## Indice
#### 1. Pulizia crawled data
#### 2.Creo una colonna con i tag puliti
#### 3.Creazione network di partenza
###### 4.1 Controllo preliminare dei nodi ed edges
#### 5. Risultati

In [1]:
import twint

import networkx as nx
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import powerlaw
warnings.filterwarnings('ignore')

from wordcloud import WordCloud
import math
from operator import itemgetter

%matplotlib inline 

### PULIZIA CRAWLED DATA

In [2]:
df = pd.read_csv("../Dataset FFF/climatechange.csv", sep='\t')

In [9]:
df.tail(2)

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
50017,1175070693391765507,1175070693391765507,2019-09-20 17:34:19 ora legale Europa occidentale,2019-09-20,17:34:19,200,19811643,lvz,LVZ,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
50018,1175070453859094528,1175070453859094528,2019-09-20 17:33:22 ora legale Europa occidentale,2019-09-20,17:33:22,200,881570054801944576,gswnature,Georgie Ward,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


In [5]:
# Get all names 
for col_name in df.columns: 
    print(col_name)

id
conversation_id
created_at
date
time
timezone
user_id
username
name
place
tweet
language
mentions
urls
photos
replies_count
retweets_count
likes_count
hashtags
cashtags
link
retweet
quote_url
video
thumbnail
near
geo
source
user_rt_id
user_rt
retweet_id
reply_to
retweet_date
translate
trans_src
trans_dest


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50019 entries, 0 to 50018
Data columns (total 36 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               50019 non-null  int64  
 1   conversation_id  50019 non-null  int64  
 2   created_at       50019 non-null  object 
 3   date             50019 non-null  object 
 4   time             50019 non-null  object 
 5   timezone         50019 non-null  int64  
 6   user_id          50019 non-null  int64  
 7   username         50019 non-null  object 
 8   name             50007 non-null  object 
 9   place            18 non-null     object 
 10  tweet            50019 non-null  object 
 11  language         50019 non-null  object 
 12  mentions         50019 non-null  object 
 13  urls             50019 non-null  object 
 14  photos           50019 non-null  object 
 15  replies_count    50019 non-null  int64  
 16  retweets_count   50019 non-null  int64  
 17  likes_count 

Toglie created_at, time, timezone, lasciando la data
Sono dati ripetuti 

In [3]:
df.drop(['trans_dest', 'trans_src','translate','retweet_date','geo','source','user_rt_id', 'user_rt','retweet_id',
        'near','place','thumbnail', 'time', 'timezone', 'video', 'photos', 'cashtags', 'created_at','quote_url','id','conversation_id','urls'], axis=1, inplace=True)

In [4]:
df.columns

Index(['date', 'user_id', 'username', 'name', 'tweet', 'language', 'mentions',
       'replies_count', 'retweets_count', 'likes_count', 'hashtags', 'link',
       'retweet', 'reply_to'],
      dtype='object')

Restingendo il range di tweet con numero di likes, retweet e replies
Si arriva a 16335 valori totali

In [5]:
df=df[df['likes_count']>=7]
df=df[df['replies_count']>=2]
df=df[df['retweets_count']>=3]

In [6]:
df.drop(df.index[df['hashtags'] == '[]'], inplace=True)

In [7]:
df.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23504 entries, 5 to 50017
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   date            23504 non-null  object
 1   user_id         23504 non-null  int64 
 2   username        23504 non-null  object
 3   name            23504 non-null  object
 4   tweet           23504 non-null  object
 5   language        23504 non-null  object
 6   mentions        23504 non-null  object
 7   replies_count   23504 non-null  int64 
 8   retweets_count  23504 non-null  int64 
 9   likes_count     23504 non-null  int64 
 10  hashtags        23504 non-null  object
 11  link            23504 non-null  object
 12  retweet         23504 non-null  bool  
 13  reply_to        23504 non-null  object
dtypes: bool(1), int64(4), object(9)
memory usage: 2.5+ MB


### Creo una colonna con i tag puliti

In [8]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mir-9\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Faccio questa funzione per pulire i tags dalle quadre, la applico a una nuova colonna (clean_tags)

In [71]:
df.head()

,clean_tags,hashtags,date,user_id,username,name,tweet,language,mentions,replies_count,retweets_count,likes_count,link,retweet,reply_to
6,fridaysforfuture,['fridaysforfuture'],2022-07-10,1096766566279073792,parents4future,Parents For Future #EndFossilFuels,"Eltern wissen, wer hier dumm ist. #FridaysForF...",de,[],21,54,595,https://twitter.com/parents4future/status/1546...,False,[]
12,fridayforfuture,['fridayforfuture'],2022-07-10,403735368,hahnflo,Florian Hahn,"Wer erklärt #FridayForFuture, dass es ohne NAT...",de,[],127,65,679,https://twitter.com/hahnflo/status/15461339384...,False,[]
13,"klimakrise, govegan, fridaysforfuture","['klimakrise', 'govegan', 'fridaysforfuture']",2022-07-10,1529883123881857031,vegancardio,Der vegane Kardiologe,"Was sind das eigentlich für Menschen, die sich...",de,[],79,25,348,https://twitter.com/VeganCardio/status/1546127...,False,[]
22,hitzewelle,['hitzewelle'],2022-07-10,364849195,bayernrocker,Andi Schrank🤘🖖🌹🌻,So sehr ihr euch übrigens über @FridayForFutur...,de,"[{'screen_name': 'fridayforfuture', 'name': 'f...",20,14,140,https://twitter.com/bayernrocker/status/154607...,False,[]
37,"fridaysforfuture, peoplenotprofit","['fridaysforfuture', 'peoplenotprofit']",2022-07-09,1449429842999525377,climateworrier3,Climate Worrier,Climate striking in school yesterday (photo ta...,en,[],9,141,497,https://twitter.com/ClimateWorrier3/status/154...,False,[]


In [9]:
def remove_content(text):
    text = re.sub(r"\[", '', text) #remove ]
    text=re.sub(r'\]','',text) #remove ]
    text=re.sub(r'\'','',text) #remove apici
    text=re.sub(r'\n','',text) #remove spazio vuoto

    return text

In [10]:
df['clean_tags']=df['hashtags'].apply(lambda x: remove_content(x))
len(df['clean_tags'])

23504

In [74]:
df.head()

,clean_tags,hashtags,date,user_id,username,name,tweet,language,mentions,replies_count,retweets_count,likes_count,link,retweet,reply_to
6,fridaysforfuture,['fridaysforfuture'],2022-07-10,1096766566279073792,parents4future,Parents For Future #EndFossilFuels,"Eltern wissen, wer hier dumm ist. #FridaysForF...",de,[],21,54,595,https://twitter.com/parents4future/status/1546...,False,[]
12,fridayforfuture,['fridayforfuture'],2022-07-10,403735368,hahnflo,Florian Hahn,"Wer erklärt #FridayForFuture, dass es ohne NAT...",de,[],127,65,679,https://twitter.com/hahnflo/status/15461339384...,False,[]
13,"klimakrise, govegan, fridaysforfuture","['klimakrise', 'govegan', 'fridaysforfuture']",2022-07-10,1529883123881857031,vegancardio,Der vegane Kardiologe,"Was sind das eigentlich für Menschen, die sich...",de,[],79,25,348,https://twitter.com/VeganCardio/status/1546127...,False,[]
22,hitzewelle,['hitzewelle'],2022-07-10,364849195,bayernrocker,Andi Schrank🤘🖖🌹🌻,So sehr ihr euch übrigens über @FridayForFutur...,de,"[{'screen_name': 'fridayforfuture', 'name': 'f...",20,14,140,https://twitter.com/bayernrocker/status/154607...,False,[]
37,"fridaysforfuture, peoplenotprofit","['fridaysforfuture', 'peoplenotprofit']",2022-07-09,1449429842999525377,climateworrier3,Climate Worrier,Climate striking in school yesterday (photo ta...,en,[],9,141,497,https://twitter.com/ClimateWorrier3/status/154...,False,[]


In [11]:
df.insert(0, 'clean_tags', df.pop('clean_tags'))

In [12]:
df.insert(1, 'hashtags', df.pop('hashtags'))

In [13]:
df.head(10)

,clean_tags,hashtags,date,user_id,username,name,tweet,language,mentions,replies_count,retweets_count,likes_count,link,retweet,reply_to
5,"fossilfuels, wind, solar, green, hydrogen, ene...","['fossilfuels', 'wind', 'solar', 'green', 'hyd...",2022-07-10,59386332,arikring,Arik Ring - Energy Engineering Expert,@ProfStrachan @ECOWARRIORSS @ChristineMilne @m...,en,"[{'screen_name': 'guytaur', 'name': 'scott. tr...",2,54,59,https://twitter.com/arikring/status/1546251358...,False,"[{'screen_name': 'ProfStrachan', 'name': 'Prof..."
6,fridaysforfuture,['fridaysforfuture'],2022-07-10,1096766566279073792,parents4future,Parents For Future #EndFossilFuels,"Eltern wissen, wer hier dumm ist. #FridaysForF...",de,[],21,54,595,https://twitter.com/parents4future/status/1546...,False,[]
7,fridaysforfuture,['fridaysforfuture'],2022-07-10,126978307,kl_stone,Klaus Steinfelder,SERVICETWEET: Zur aktuellen Hetze der BLÖD-Ze...,de,[],2,5,8,https://twitter.com/Kl_Stone/status/1546221657...,False,[]
12,fridayforfuture,['fridayforfuture'],2022-07-10,403735368,hahnflo,Florian Hahn,"Wer erklärt #FridayForFuture, dass es ohne NAT...",de,[],127,65,679,https://twitter.com/hahnflo/status/15461339384...,False,[]
13,"klimakrise, govegan, fridaysforfuture","['klimakrise', 'govegan', 'fridaysforfuture']",2022-07-10,1529883123881857031,vegancardio,Der vegane Kardiologe,"Was sind das eigentlich für Menschen, die sich...",de,[],79,25,348,https://twitter.com/VeganCardio/status/1546127...,False,[]
22,hitzewelle,['hitzewelle'],2022-07-10,364849195,bayernrocker,Andi Schrank🤘🖖🌹🌻,So sehr ihr euch übrigens über @FridayForFutur...,de,"[{'screen_name': 'fridayforfuture', 'name': 'f...",20,14,140,https://twitter.com/bayernrocker/status/154607...,False,[]
31,fridaysforfuture,['fridaysforfuture'],2022-07-09,1357044621050867714,freddy14792713,Freddy,#FridaysForFuture Dann zeigt mal was ihr drauf...,de,[],2,18,37,https://twitter.com/Freddy14792713/status/1545...,False,[]
37,"fridaysforfuture, peoplenotprofit","['fridaysforfuture', 'peoplenotprofit']",2022-07-09,1449429842999525377,climateworrier3,Climate Worrier,Climate striking in school yesterday (photo ta...,en,[],9,141,497,https://twitter.com/ClimateWorrier3/status/154...,False,[]
38,"climatecrisis, climatestrike, climateaction, c...","['climatecrisis', 'climatestrike', 'climateact...",2022-07-09,1101867372221288448,dmugalizi,Mugalizi Dianey,Climate justice approaches promote transformat...,en,[],3,7,11,https://twitter.com/DMugalizi/status/154571399...,False,[]
44,"peoplepower, actnow, climateaction, climatecri...","['peoplepower', 'actnow', 'climateaction', 'cl...",2022-07-09,1442093243835969540,greenagewarrior,Green Age Warriors,The older generation has failed and it is the ...,en,[],3,17,35,https://twitter.com/Greenagewarrior/status/154...,False,[]


In [14]:
#Salvo le modifiche nel df nuovo
df.to_csv('../Dataset FFF/fff.csv', index=False)

###  Creazione network di partenza

In [15]:
df1=pd.read_csv("../Dataset FFF/fff.csv")
df1.head(5)

,clean_tags,hashtags,date,user_id,username,name,tweet,language,mentions,replies_count,retweets_count,likes_count,link,retweet,reply_to
0,"fossilfuels, wind, solar, green, hydrogen, ene...","['fossilfuels', 'wind', 'solar', 'green', 'hyd...",2022-07-10,59386332,arikring,Arik Ring - Energy Engineering Expert,@ProfStrachan @ECOWARRIORSS @ChristineMilne @m...,en,"[{'screen_name': 'guytaur', 'name': 'scott. tr...",2,54,59,https://twitter.com/arikring/status/1546251358...,False,"[{'screen_name': 'ProfStrachan', 'name': 'Prof..."
1,fridaysforfuture,['fridaysforfuture'],2022-07-10,1096766566279073792,parents4future,Parents For Future #EndFossilFuels,"Eltern wissen, wer hier dumm ist. #FridaysForF...",de,[],21,54,595,https://twitter.com/parents4future/status/1546...,False,[]
2,fridaysforfuture,['fridaysforfuture'],2022-07-10,126978307,kl_stone,Klaus Steinfelder,SERVICETWEET: Zur aktuellen Hetze der BLÖD-Ze...,de,[],2,5,8,https://twitter.com/Kl_Stone/status/1546221657...,False,[]
3,fridayforfuture,['fridayforfuture'],2022-07-10,403735368,hahnflo,Florian Hahn,"Wer erklärt #FridayForFuture, dass es ohne NAT...",de,[],127,65,679,https://twitter.com/hahnflo/status/15461339384...,False,[]
4,"klimakrise, govegan, fridaysforfuture","['klimakrise', 'govegan', 'fridaysforfuture']",2022-07-10,1529883123881857031,vegancardio,Der vegane Kardiologe,"Was sind das eigentlich für Menschen, die sich...",de,[],79,25,348,https://twitter.com/VeganCardio/status/1546127...,False,[]


In [29]:
#Ti crea un dizionario in cui le chiavi sono i tag singoli e i valori sono gli id degli user
tags_dict = {}
for i in range(len(df1)):
    ll = df1["clean_tags"][i].split(",")
    for tg in ll:
        tgs=tg.lstrip()
        if tgs not in tags_dict:
            tags_dict[tgs] = [df1["tweet"][i]]
        else:
            tags_dict[tgs].append(df1["tweet"][i])

In [30]:
#Crea la lista di tags dalle chiavi del dizionario e li associa alla variabile tag
tags = list(tags_dict.keys())
#crea una lista costituita da coppie di tags
couple_tags = []
for i in range(len(tags)):
    for j in tags[i+1:]:
        couple_tags.append((tags[i], j)) 

In [31]:
len(couple_tags)

56323191

In [32]:
len(tags)

10614

In [33]:
source = []
target = []
count = []
nodes = []
for i in couple_tags:
    ll = list(set(tags_dict[i[0]]) & set(tags_dict[i[1]]))
    if len(ll)>0:
        source.append(i[0])
        target.append(i[1])
        count.append(len(ll))
        nodes.append(i[0])
        nodes.append(i[1])

In [34]:
nodes = list(set(nodes)) #list()crea una lista; set(nodes) prende i nodi senza ripetizioni

In [35]:
len(nodes)

10361

In [36]:
new_df = pd.DataFrame({"Id": nodes})
new_df.to_csv("../Dataset FFF/nodes.csv", index=False)

In [37]:
new_df = pd.DataFrame({"Source": source,
                       "Target": target,
                      "Count": count})
new_df.to_csv("../Dataset FFF/edges.csv", index=False)

qui abbiamo salvato i dati di edges e nodi ma la cosa che facciamo dopo con nx prende in input gli edges e ne costruisce un network, non so perche però otteniamo un numwero di nodi diversi

## Risultato pulizia:

In [38]:
path = "../Dataset FFF/edges.csv"
path_nodes = "../Dataset FFF/nodes.csv"

In [39]:
G = nx.read_edgelist(path, comments = "S", delimiter = ",", data=(("Count", int),))

In [40]:
print(nx.info(G))

Graph with 10361 nodes and 63960 edges


### Degree


In [41]:
fff = pd.read_csv(path)
fff1 = pd.read_csv(path_nodes)

In [42]:
fff.sort_values('Count', ascending=False)


,Source,Target,Count
3555,fridaysforfuture,climatestrike,3641
598,climateaction,fridaysforfuture,1394
3592,fridaysforfuture,climatestrikeonline,1279
2485,climatecrisis,fridaysforfuture,1145
3594,fridaysforfuture,schoolstrike4climate,1051
...,...,...,...
25945,energytransition,diesel,1
25946,energytransition,trucks,1
25949,energytransition,coventry,1
25950,energytransition,exxonknew,1


In [43]:
fff1

,Id
0,foreverychild
1,wirecard
2,sayhername
3,healthy
4,nichtmeinerbe
...,...
10356,fistingforfreedom
10357,100ee30
10358,paracaidista
10359,climatstrike


In [44]:
degree_dict = dict(G.degree(G.nodes()))
nx.set_node_attributes(G, degree_dict, 'degree')
sorted_degree = sorted(degree_dict.items(), key = itemgetter(1), reverse = True)
print(" ---- Top 20 nodes by degree ----")
for d in sorted_degree[:20]:
    print("\t" + str(d))

 ---- Top 20 nodes by degree ----
	('fridaysforfuture', 8976)
	('climatestrike', 2081)
	('climateaction', 1302)
	('climatecrisis', 1077)
	('climatechange', 1034)
	('fridayforfuture', 1032)
	('climateemergency', 913)
	('klimakrise', 698)
	('klimaschutz', 675)
	('fff', 659)
	('gretathunberg', 653)
	('klimastreik', 639)
	('climatestrikeonline', 618)
	('climatejustice', 537)
	('extinctionrebellion', 534)
	('climateactionnow', 532)
	('climate', 514)
	('schoolstrike4climate', 507)
	('klimawandel', 477)
	('fridaymotivation', 428)
